## ___Toxic Tweets Fine-Tuned Pretrained Transformer with Multi Head Classifier___

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset 
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

In [ ]:
# Getting training dataset
df = pd.read_csv("../data/raw/train.csv")
# Comments as list of strings for training texts
texts = df["comment_text"].tolist()
# Labels extracted from dataframe as list of lists
labels = df[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]].values.tolist()
# Training set split into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)

In [ ]:
# Tokenizing training and validation sets
class ToxicTweetsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, index):
        input_ids = self.encodings['input_ids'][index]
        attention_mask = self.encodings['attention_mask'][index]
        labels = torch.tensor(self.labels[index], dtype=torch.float32)

        return {'input_ids': input_ids,
                'attention_mask': attention_mask,
                'labels': labels}


In [ ]:
# Choosing base pretrained model
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

# Tokenizing and encoding training and validation sets
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Creating datasets for training and validation
train_dataset = ToxicTweetsDataset(train_encodings, train_labels)
val_dataset = ToxicTweetsDataset(val_encodings, val_labels)

# Setting training arguments
training_args = TrainingArguments(
    output_dir="../models/fine_tuned",
    num_train_epochs=2, 
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=64, 
    warmup_steps=500,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=1
)

# If GPU is available, use it, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Creating model
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=6)

# Creating trainer
trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Training model
trainer.train()